# Interactive Data Exploration

## Interactions in the Jupyter Notebook Environment

### DATA 601


Further Reading:

In this notebook, we'll use Jupyter Widgets (ipywidgets) in conjunction with plotly to make data exploration *more* interactive. Jupyter Widgets provides a set of interactive widgets that can be used in the IPython environment. 

As always, please consult the official documentation for more details:<br>
  - [Jupyter Widgets User Guide](https://ipywidgets.readthedocs.io/en/7.x/)
  - [Plotly FigureWidget](https://plotly.com/python/figurewidget/)
  
 

In [1]:
import ipywidgets as widgets
from ipywidgets import interact
import plotly.graph_objs as go

import numpy as np
import datetime as dt
import pandas as pd

#print(widgets.version_info)

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


## Basic Widget Mechanism

- Widgets work with callback functions that are triggered in reponse to events (buttons, sliders, dropdown selections etc.).
<br><br>
- To work with a widget, follow these steps:
  1. Choose a widget with appropriate controls.
  2. Register a callback function with the widget.
  
  The above steps can be combined with the help of convenient Python syntactic sugar ([decorators](https://realpython.com/primer-on-python-decorators/)).
<br><br>
- When an event is triggered, the widget will call the registered callback function with appropriate arguments. Thus a widget alongwith its registered callback function can be used for interactive tasks such as interactive queries, updating plots, exploring the effect of parameters etc.

## Example 1: Updating a Plotly Plot via a Slider

- Consider the following cosine wave:
  $$
    f(x,m,t) = \cos [2\,\pi \,m \,(x-t)]
  $$
<br><br>  
- Let's plot it on the interval $[0,1]$ and explore the effect of the parameters $m$ and $t$ via sliders.

In [2]:
# Function to compute
def wave(x, m=1, t=0):
    return np.cos(2 * np.pi * m * (x - t))

# Build a figure which can then be updated via a callback
x = np.linspace(0,1,256)
scatt = go.Scatter(x=x, y=wave(x))
# Note the use of FigureWidget rather than Figure
fig_wave = go.FigureWidget(data=scatt) 

# Interaction widget and callback via a decorator
# Tuples passed to the interact function give sliders.
# See https://ipywidgets.readthedocs.io/en/latest/examples/Using%20Interact.html#Widget-abbreviations

@interact(m=(0,12,1), t=(0,1, 0.01))
def update(m=1, t=0):
    fig_wave.update_traces(y=wave(x,m,t))
    

# Display the Figure. Do not use .show() as that uses a different renderer.
# see https://plot.ly/python/renderers/ for details
fig_wave

## Example 2: Year Selection via a Dropdown Interaction 

- Recall the Calgary Rainfall Dataset. First we'll wrangle it into a format that is suitable for monthly/yearly aggregations.
<br><br>
- We can then execute a query and update the results in a plot interactively.

In [ ]:
# Load the rainfall dataset and wrangle it into a suitable format with 
# years and months as rows (multi-index) and channels as columns

# Please adjust path appropriately.
rdata = pd.read_csv("Historical_Rainfall_20240201.csv", 
                    parse_dates=['TIMESTAMP'], date_format="%Y/%m/%d %I:%M:%S %p")

# Strip out the year and month from the 'YEAR' and 'TIMESTAMP'
# fields and build a new DataFrame with this information.
#
yseries = rdata['YEAR']
mseries = rdata['TIMESTAMP'].map(lambda x: x.month)

rdata = pd.DataFrame({'CHANNEL' : rdata['CHANNEL'], 'YEAR' : yseries, 'MONTH' : mseries, 'RAINFALL' : rdata['RAINFALL']})

rdata = rdata.groupby(['YEAR', 'MONTH','CHANNEL']).sum().unstack()
display(rdata.head())
display(rdata.tail())

RAINFALL                                                    ...  \
CHANNEL          1     2   3     4     5     6   7     8     9     10  ...   
YEAR MONTH                                                             ...   
1988 5          NaN   NaN NaN   0.6   NaN   4.0 NaN   5.4   5.6   NaN  ...   
     6          NaN   NaN NaN   NaN   6.2   1.8 NaN  14.4  28.8  11.8  ...   
     7          NaN   NaN NaN   NaN  28.2  21.4 NaN  23.2  32.4  15.0  ...   
     8          NaN   1.0 NaN  34.4  59.2  30.6 NaN  95.2  96.6  28.4  ...   
     9          NaN  35.2 NaN  32.4   NaN  39.4 NaN  45.8  40.6  30.4  ...   

                                                    
CHANNEL     40  41  42  43  44  45  46  47  48  99  
YEAR MONTH                                          
1988 5     NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN  
     6     NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN  
     7     NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN  
     8     NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN  
     9     NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN  

[5 rows x 47 columns]

RAINFALL                                                         \
CHANNEL          1      2     3     4     5     6     7     8     9     10   
YEAR MONTH                                                                   
2023 5         39.0   29.0  38.2  17.8  23.8  32.0  28.6  20.8  25.0  27.0   
     6         81.6   62.4  80.4  72.2  56.6  63.0  82.8  77.2  73.8  75.4   
     7         70.4  112.2  53.6  97.0  81.4  67.6  79.6  77.4  18.8  47.8   
     8         44.0   47.4  58.6  45.4  42.6  82.8  51.2  54.6  37.2  46.8   
     9         13.0   13.2  14.2  12.2  11.6  16.0  13.6  20.8  17.0  15.0   

            ...                                                          
CHANNEL     ...    40    41    42    43    44  45    46    47    48  99  
YEAR MONTH  ...                                                          
2023 5      ...  28.2  36.8  24.2  17.2   8.8 NaN  30.8  49.8  19.2 NaN  
     6      ...  78.2  73.4  77.6  88.6  98.8 NaN  58.6  67.6  68.6 NaN  
     7      ...  48.2  20.8  32.0  37.6  26.0 NaN  39.6  37.0  23.2 NaN  
     8      ...  52.6  48.4  55.0  49.4  49.2 NaN  37.8  44.0  52.6 NaN  
     9      ...  17.8   4.0  34.8  24.6  16.2 NaN  14.6  29.0  24.6 NaN  

[5 rows x 47 columns]

In [ ]:
# Now let's explore the data with the help of a drop-down interactor.

# Build a bar chart that we'll then update with the help of a call-back function
bar = go.Bar()
fig_bar = go.FigureWidget(data=bar)

# Specify range of the y-axis for comparison between different years
fig_bar.update_yaxes(range=[0, 250])

month_labels = {5:'May', 6:'Jun', 7:'Jul', 8:'Aug', 9:'Sep'}

# A list passed to interact() will yield a drop-down interactor
# Or we can pass a tuple to get a slider
@interact(year=[*range(2023,1988,-1)])
def update_bar(year=2023):
    data = rdata.loc[year].mean(axis=1,skipna=True)
    fig_bar.update_traces(x=pd.Series(data.index.values).map(month_labels).values,
                          y=data.values)
    fig_bar.update_layout(title_text="YYC total monthly rainfall for {0}".format(year))

fig_bar

interactive(children=(Dropdown(description='year', options=(2023, 2022, 2021, 2020, 2019, 2018, 2017, 2016, 20…

FigureWidget({
    'data': [{'type': 'bar',
              'uid': 'ed3e507f-44c6-4c54-b95e-330d21854214',
              'x': array(['May', 'Jun', 'Jul', 'Aug', 'Sep'], dtype=object),
              'y': array([27.27272727, 76.54545455, 50.09545455, 49.87727273, 19.65454545])}],
    'layout': {'template': '...', 'title': {'text': 'YYC total monthly rainfall for 2023'}, 'yaxis': {'range': [0, 250]}}
})

## **All Possible `@interact` Input Types and Their Widgets**

Here is the **complete list** of all possible **`@interact` input types** and their corresponding widgets in **ipywidgets**:

| **Parameter Type**       | **Example Syntax**                            | **Widget Created**  |
|-------------------------|-----------------------------------------------|--------------------|
| **Integer range**       | `@interact(value=(0, 100, 1))`               | 🎚️ **Integer Slider** (min=0, max=100, step=1) |
| **Float range**         | `@interact(value=(0.0, 10.0, 0.1))`          | 🎚️ **Float Slider** (min=0.0, max=10.0, step=0.1) |
| **List of values**      | `@interact(option=['A', 'B', 'C'])`          | 📌 **Dropdown Menu** (static list) |
| **Dictionary mapping**  | `@interact(option={'Low': 1, 'High': 2})`    | 📌 **Dropdown Menu (Labeled Options)** |
| **Boolean (True/False)**| `@interact(flag=True)`                        | ✅ **Checkbox** (Toggle on/off) |
| **String input**        | `@interact(name="Enter your name")`          | ✏️ **Textbox** (User input) |
| **Dropdown with Tuple** | `@interact(choice=(0, 5, 1))`                | 📌 **Dropdown with range** |
| **Multiple Selection (List)** | `@interact(color=['Red', 'Blue', 'Green'])` | 📌 **Dropdown with single selection** |
| **Multiple Selection (Tuple)** | `@interact(choice=(1, 10, 1))` | 📌 **Dropdown from a numeric range** |
| **Dropdown with Integer Keys** | `@interact(level={1: 'Beginner', 2: 'Intermediate', 3: 'Advanced'})` | 📌 **Dropdown with numeric keys** |
| **Password Input** | `widgets.Password(value='mypassword')` | 🔒 **Password input (masked)** |
| **Textarea Input** | `widgets.Textarea(value="Enter text here")` | 📝 **Multi-line text input** |
| **Date Picker** | `widgets.DatePicker(description="Pick a date")` | 📅 **Date selection widget** |
| **Time Picker** | `widgets.TimePicker(description="Pick a time")` | ⏰ **Time selection widget** |
| **Color Picker** | `widgets.ColorPicker(concise=True)` | 🎨 **Color selection widget** |
| **Button with Action** | `widgets.Button(description="Click me!")` | 🔘 **Button (Triggers an action)** |
| **File Upload** | `widgets.FileUpload(accept='.csv', multiple=True)` | 📂 **File Upload Widget** |

---
